In [6]:
!pip install SPARQLWrapper


# **Popolazione del CSV con informazioni ricavate dai dati richies**

In [12]:
import csv
import datetime
from SPARQLWrapper import SPARQLWrapper, JSON

def query_author_data(author_name):
    """Interroga Wikidata per ottenere informazioni sull'autore."""
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = f"""
    SELECT ?genderLabel ?birthYear WHERE {{
        ?author ?label "{author_name}"@en.
        ?author wdt:P21 ?gender;
               wdt:P569 ?birthDate.
        BIND(YEAR(?birthDate) AS ?birthYear)

        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    if results["results"]["bindings"]:
        result = results["results"]["bindings"][0]
        gender = result["genderLabel"]["value"]
        birth_year = int(result["birthYear"]["value"])
        return gender, birth_year
    else:
        return "Unknown", "Unknown"

def enrich_csv(input_file, output_file):
    # Leggi il file CSV di input
    with open(input_file, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        rows = list(reader)

    # Campi aggiuntivi
    fieldnames = reader.fieldnames + ["Gender", "Author Birth Year"]

    # Scrivi il nuovo file CSV
    with open(output_file, mode='w', encoding='utf-8', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in rows:
            author = row["Nome dell'autore dell'opera"]
            gender, birth_year = query_author_data(author)

            row["Gender"] = gender
            row["Author Birth Year"] = birth_year

            writer.writerow(row)

# Esempio di utilizzo
enrich_csv(
    input_file="Argomento Opera (Risposte) - Risposte del modulo 1.csv",
    output_file="work_topics.csv"
)

In [14]:
!pip install pysparql-anything


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 35.8 MB/s eta 0:00:00
Welcome to PySPARQL Anything 0.9.0.3, the SPARQL Anything Python library.
No SPARQL Anything executable jar has been found in the installation folder.
Proceeding to download the SPARQL Anything 0.9.0 jar:


100%|██████████| 198M/198M [00:03<00:00, 62.8MiB/s]


The Download was successful!
The system is now ready for use!


In [ ]:
import pysparql_anything as sa

engine = sa.SparqlAnything()

engine.run(query='./queries/author.sparql',output='./output/ex_author.ttl',format='ttl')